In [87]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import plotly.express as px
import branca.colormap as cm
from IPython.display import HTML
from folium.plugins import MousePosition
import warnings
from folium import Popup
warnings.filterwarnings('ignore')

In [88]:
data_collar = pd.read_csv("datasets/Collar.csv")
data_litologi = pd.read_csv("datasets/Litologi.csv")
data_assay = pd.read_csv("datasets/Assay.csv")

In [89]:
data_collar['Hole_id'] = data_collar['Hole_id'].str.extract('(\d+)', expand=False).astype(int)
data_assay['Hole_id'] = data_assay['Hole_id'].str.extract('(\d+)', expand=False).astype(int)
data_litologi['hole_id'] = data_litologi['hole_id'].str.extract('(\d+)', expand=False).astype(int)

In [90]:
# data_collar['X'] = data_collar['X'].astype(float)
# data_collar['Y'] = data_collar['Y'].astype(float)
# # data_collar['Hole_id'] = data_collar['Hole_id'].str.extract('(\d+)', expand=False).astype(int)

In [ ]:
new_data = pd.concat([
    data_assay[['Hole_id']],
    data_litologi[['dept_from', 'Dept_to', 'Litho']].rename(columns={
        'dept_from': 'Depth_from',
        'Dept_to': 'Depth_to'
    }),
    data_assay[[' Ni']].rename(columns={' Ni': 'Ni'})
], axis=1)

# save csv
new_data.to_csv('datasets/new_data.csv', index=False)



,Hole_id,Depth_from,Depth_to,Litho,Ni
0,1.0,0.0,1.0,LIM,0.50
1,1.0,1.0,2.0,LIM,0.72
2,1.0,2.0,3.0,LIM,0.89
3,1.0,3.0,4.0,LIM,0.98
4,1.0,4.0,5.0,LIM,0.90


In [ ]:
sap_data = new_data[new_data['Litho'] == 'SAP'].copy()
sap_data['Tebal_Per_Sampel'] = sap_data['Depth_to'] - sap_data['Depth_from']
sap_data.to_csv('datasets/sap_data.csv', index=False)

,Hole_id,Depth_from,Depth_to,Litho,Ni,Tebal_Per_Sampel
11,1.0,11.00,12.00,SAP,1.65,1.00
12,1.0,12.00,13.00,SAP,2.08,1.00
13,1.0,13.00,14.00,SAP,1.59,1.00
14,1.0,14.00,14.75,SAP,1.74,0.75
15,1.0,14.75,15.00,SAP,1.03,0.25
...,...,...,...,...,...,...
3208,125.0,6.00,7.00,SAP,0.94,1.00
3209,125.0,7.00,8.00,SAP,0.95,1.00
3210,125.0,8.00,9.00,SAP,1.12,1.00
3211,125.0,9.00,10.00,SAP,1.11,1.00


In [ ]:
sap_data['Weighted_Ni'] = sap_data['Ni'] * sap_data['Tebal_Per_Sampel']

,Hole_id,Depth_from,Depth_to,Litho,Ni,Tebal_Per_Sampel,Weighted_Ni
11,1.0,11.00,12.00,SAP,1.65,1.00,1.6500
12,1.0,12.00,13.00,SAP,2.08,1.00,2.0800
13,1.0,13.00,14.00,SAP,1.59,1.00,1.5900
14,1.0,14.00,14.75,SAP,1.74,0.75,1.3050
15,1.0,14.75,15.00,SAP,1.03,0.25,0.2575
...,...,...,...,...,...,...,...
3208,125.0,6.00,7.00,SAP,0.94,1.00,0.9400
3209,125.0,7.00,8.00,SAP,0.95,1.00,0.9500
3210,125.0,8.00,9.00,SAP,1.12,1.00,1.1200
3211,125.0,9.00,10.00,SAP,1.11,1.00,1.1100


In [94]:
aggregated_data = sap_data.groupby('Hole_id').agg(
    Depth_from = ('Depth_from', 'min'),
    Depth_to = ('Depth_to', 'max'),
    Total_SAP = ('Tebal_Per_Sampel', 'sum'),
    Min_Ni = ('Ni', 'min'),
    Max_Ni = ('Ni', 'max')
).reset_index()

In [95]:
weighted_sum = sap_data.groupby('Hole_id')['Weighted_Ni'].sum().reset_index(name='Weighted_Sum_Ni')

In [ ]:
aggregated_data = pd.merge(aggregated_data, weighted_sum, on='Hole_id')

,Hole_id,Depth_from,Depth_to,Total_SAP,Min_Ni,Max_Ni,Weighted_Sum_Ni
0,1.0,11.0,16.00,5.00,1.03,2.08,8.4025
1,2.0,9.0,24.00,15.00,1.38,3.07,34.6830
2,3.0,4.4,9.00,4.60,0.83,2.53,8.8120
3,4.0,8.0,13.00,5.00,1.25,2.17,8.0700
4,5.0,5.0,10.00,5.00,1.50,2.13,8.6200
...,...,...,...,...,...,...,...
120,121.0,10.0,17.00,7.00,1.18,2.12,11.4412
121,122.0,9.0,11.33,2.33,0.66,2.10,3.4898
122,123.0,8.0,14.00,6.00,1.32,3.92,15.5682
123,124.0,8.0,18.00,10.00,0.60,2.93,19.7082


In [ ]:
aggregated_data['Avg_Ni'] = aggregated_data['Weighted_Sum_Ni'] / aggregated_data['Total_SAP']

,Hole_id,Depth_from,Depth_to,Total_SAP,Min_Ni,Max_Ni,Weighted_Sum_Ni,Avg_Ni
0,1.0,11.0,16.00,5.00,1.03,2.08,8.4025,1.680500
1,2.0,9.0,24.00,15.00,1.38,3.07,34.6830,2.312200
2,3.0,4.4,9.00,4.60,0.83,2.53,8.8120,1.915652
3,4.0,8.0,13.00,5.00,1.25,2.17,8.0700,1.614000
4,5.0,5.0,10.00,5.00,1.50,2.13,8.6200,1.724000
...,...,...,...,...,...,...,...,...
120,121.0,10.0,17.00,7.00,1.18,2.12,11.4412,1.634457
121,122.0,9.0,11.33,2.33,0.66,2.10,3.4898,1.497768
122,123.0,8.0,14.00,6.00,1.32,3.92,15.5682,2.594700
123,124.0,8.0,18.00,10.00,0.60,2.93,19.7082,1.970820


In [ ]:
aggregated_data.drop(columns=['Weighted_Sum_Ni'], inplace=True)
aggregated_data.to_csv('datasets/aggregated_data.csv', index=False) 

,Hole_id,Depth_from,Depth_to,Total_SAP,Min_Ni,Max_Ni,Avg_Ni
0,1.0,11.0,16.0,5.0,1.03,2.08,1.680500
1,2.0,9.0,24.0,15.0,1.38,3.07,2.312200
2,3.0,4.4,9.0,4.6,0.83,2.53,1.915652
3,4.0,8.0,13.0,5.0,1.25,2.17,1.614000
4,5.0,5.0,10.0,5.0,1.50,2.13,1.724000


In [99]:
# # Filter data untuk litologi SAP
# sap_data = new_data[new_data['Litho'] == 'SAP'].copy()

# # Hitung ketebalan per baris
# sap_data['Thickness'] = sap_data['Depth_to'] - sap_data['Depth_from']

# # Hitung total ketebalan, min, dan max Ni per Hole_id
# summary = sap_data.groupby('Hole_id').agg(
#     Depth_from=('Depth_from', 'min'),
#     Depth_to=('Depth_to', 'max'),
#     Total_SAP=('Thickness', 'sum'),
#     Min_Ni=('Ni', 'min'),
#     Max_Ni=('Ni', 'max')
# ).reset_index()

# # Hitung total tertimbang Ni per Hole_id
# sap_data['Weighted_Ni'] = sap_data['Ni'] * sap_data['Thickness']
# weighted_sum = sap_data.groupby('Hole_id')['Weighted_Ni'].sum().reset_index(name='Weighted_Sum_Ni')

# # Gabungkan dengan summary
# aggregated_data = pd.merge(summary, weighted_sum, on='Hole_id')

# # Hitung rata-rata tertimbang
# aggregated_data['Avg_Ni'] = aggregated_data['Weighted_Sum_Ni'] / aggregated_data['Total_SAP']

# # Hapus kolom antara jika tidak ingin disimpan
# aggregated_data.drop(columns=['Weighted_Sum_Ni'], inplace=True)

# # Simpan ke file
# aggregated_data.to_csv('datasets/aggregated_data.csv', index=False)

# # Tampilkan hasil
# aggregated_data.head()


In [ ]:
# Combine all relevant columns for plotting data
plot_data = pd.concat([
    aggregated_data[['Hole_id']],
    data_collar[['X', 'Y']],
    aggregated_data[['Depth_from', 'Depth_to', 'Total_SAP', 'Avg_Ni', 'Min_Ni', 'Max_Ni']]
], axis= 1)

# save to .csv file
plot_data.to_csv('datasets/plot_data.csv', index=False)


,Hole_id,X,Y,Depth_from,Depth_to,Total_SAP,Avg_Ni,Min_Ni,Max_Ni
0,1.0,-2.507508,121.513665,11.0,16.00,5.00,1.680500,1.03,2.08
1,2.0,-2.507511,121.512784,9.0,24.00,15.00,2.312200,1.38,3.07
2,3.0,-2.507511,121.511902,4.4,9.00,4.60,1.915652,0.83,2.53
3,4.0,-2.507515,121.510981,8.0,13.00,5.00,1.614000,1.25,2.17
4,5.0,-2.507488,121.510041,5.0,10.00,5.00,1.724000,1.50,2.13
...,...,...,...,...,...,...,...,...,...
120,121.0,-2.510636,121.507853,10.0,17.00,7.00,1.634457,1.18,2.12
121,122.0,-2.510652,121.508255,9.0,11.33,2.33,1.497768,0.66,2.10
122,123.0,-2.510583,121.508693,8.0,14.00,6.00,2.594700,1.32,3.92
123,124.0,-2.510650,121.509187,8.0,18.00,10.00,1.970820,0.60,2.93


In [101]:
# plt.figure(figsize=(15, 8))
# plt.scatter(data_collar['Y'], data_collar['X'], c='blue', alpha=0.5)
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.title('Peta titik dari data collar')
# plt.grid(True)
# plt.show()

In [102]:
# Define bounds for map view
min_lat, max_lat = plot_data['X'].min(), plot_data['X'].max()
min_lon, max_lon = plot_data['Y'].min(), plot_data['Y'].max()

# Define value ranges for color mapping
Min_Avg_Ni = plot_data['Avg_Ni'].min()
Max_Avg_Ni = plot_data['Avg_Ni'].max()

# Create base map
map = folium.Map(
    zoom_start=17,
    min_zoom=10,
    control_scale= True,
    font_size='1.5rem'
    )
# Fitting map to data bounds
map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

# Create color scale
colormap = cm.linear.YlOrRd_09.scale(Min_Avg_Ni, Max_Avg_Ni)
colormap.caption = ' Avg. Nickel Value Scale'
colormap.add_to(map)

# Add mouse position to bottom right of map
mouse_position = MousePosition(
    position='bottomright',
    separator=' : ',
    prefix='Lat | Lon:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 10);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 10);}",
)
mouse_position.add_to(map)

# Add circle markers for each row
for _, row in plot_data.iterrows():
    # color for the circle
    color = colormap(row['Avg_Ni'])
    # radius size of each circle
    radius = 2 + row['Total_SAP'] * 0.7

    folium.CircleMarker(
        location=[row['X'], row['Y']],  
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=1,
        tooltip=f"<b>BH:</b> {row['Hole_id']:.0f}<br>",
        popup=Popup(
        html=(
            f"<b>BH:</b> {row['Hole_id']:.0f}<br>"
            f"<b>Lat:</b> {row['X']:.5f}<br>"
            f"<b>Lon:</b> {row['Y']:.5f}<br>"
            f"<b>SAP:</b> {row['Total_SAP']:.2f} meter<br>"
            f"<b>Avg. Ni:</b> {row['Avg_Ni']:.2f}<br>"
            f"<b>Min. Ni:</b> {row['Min_Ni']:.2f}<br>"
            f"<b>Max. Ni:</b> {row['Max_Ni']:.2f}<br>"
        ),
        max_width=200  
    )
    ).add_to(map)

# Displaying the map
# The map will fill the entire output cell, so it's customized through css
display(HTML('<div style="width: 1000px; height: 700px;">'+map._repr_html_()+'</div>'))
# map # uncomment this to use show full size of the map

In [103]:
# map = folium.Map(zoom_starts = 15)

# locations = []
# for _, row in data_collar.iterrows():
#     location = [row['X'], row['Y']]
#     locations.append(location)
#     folium.CircleMarker(location=location, radius=3, color='blue', fill=True, fill_opacity=0.7).add_to(map)

# # Fit bounds ke semua lokasi
# map.fit_bounds(locations)
# map.save("collar_map.html")

In [104]:
# fig = px.scatter_mapbox(
#     data_collar,
#     lat="X",
#     lon="Y",
#     zoom=16,
#     height=600
# )

# fig.update_layout(mapbox_style="open-street-map")

# fig.show()